In [172]:
import pandas as pd
import sys
import os
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if path not in sys.path:
    sys.path.append(path)
from constants import NER
# get the first key in NERa
from fuzzywuzzy import fuzz
from constants import NER
from fuzzywuzzy import fuzz
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
# import sklearn_crfsuite
import sklearn_crfsuite

In [55]:
# df = pd.read_csv('../data/data_for_hajez_classifier.csv')

In [56]:
# df = df.sample(1000)

In [57]:
# from joblib import Parallel, delayed
# import multiprocessing

In [58]:
# def process_word(word, NER):
#     for key in NER.keys():
#         if any(fuzz.ratio(word, k) >= 80 for k in key):
#             return NER[key]
#     return "O"

In [59]:
# def generate_ner_string(text, NER):
#     num_cores = multiprocessing.cpu_count()
#     words = text.split()
#     results = Parallel(n_jobs=num_cores)(
#         delayed(process_word)(word, NER) for word in words)
#     return " ".join(results)

In [60]:
# df['ner'] = Parallel(n_jobs=-1)(
#     delayed(generate_ner_string)(text, NER) for text in df['full_text'])

In [61]:
# # get any row contains "جيش" in the text
# df[df['full_text'].str.contains('اسرائيل')]

In [62]:
# df.to_csv('../data/data_with_ner.csv2', index=False)

In [173]:
df = pd.read_csv('../data/data_with_ner.csv2')

In [174]:
# drop any row the full_text is " "
df = df[df['full_text'] != " "]

In [175]:
# convert full_text and ner to str
df['full_text'] = df['full_text'].astype(str)
df['ner'] = df['ner'].astype(str)

In [176]:
X = df['full_text']
y = df['ner']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [177]:
import pyarabic.araby as araby
import pyarabic.araby as araby
def is_punct(word):
    """
    Returns True if the word is punctuation, False otherwise.
    """
    # check if the word is punctuation
    punct_chars = set('؟،؛?!.,:;')
    return all(char in punct_chars for char in word)

In [178]:
def is_it_bar(word):
    """
    Returns True if the word is punctuation, False otherwise.
    """
    from fuzzywuzzy import fuzz
    barr_words = ["دوار","حاجز","طريق","مخصوم","محسوم"]
    return any((fuzz.ratio(word, k) >= 80 for k in barr_words))
    
def is_it_status(word):
    stat_words = ["ازمه","سهلة","فاضيه","سالك","فاتحة","ماشي","ازمة","فاتح","مسكر","فتح","متوقف","تفتيش","مغلق","سااالكه",
            "سالك","سالكة","فاتح","فاتحة","فاضي","مفتوح","فتح","خرجت","ماشية","ماشيه","فش اشي", "سالك","سألك","نظيف","نظيفة",
            "فاضي","نظيفه","مسكر","مغلق","مغلقة","اغلاق","سكرو","زاطمة","زاطم","سكر","زطم","واقف","ازمة", "أزمة", "ازمه", "تشديد",
            "تفتيش""شرطه ","مستوطن","شرطة","جيش","يهود","حذر"]
    return any((fuzz.ratio(word, k) >= 80 for k in stat_words))
    

In [187]:
def combine_sub_words(output):
    for index in range(len(output)):
        if index < len(output) - 1:
            if '##' in output[index+1]['word']:
                output[index]['word'] = output[index]['word'] + output[index+1]['word']
                output[index]['word'] = output[index]['word'].replace('##', '')
                # delete the next word
                output.pop(index+1)
    return output

In [189]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Load the pre-trained model and tokenizer
model_name = "CAMeL-Lab/bert-base-arabic-camelbert-ca-pos-egy"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define the pipeline
nlp = pipeline("ner", model=model_name, tokenizer=tokenizer)

# # Encode a single input sequence
# input_sequence = "اليوم مريت على حاجز حوارة ولقيتو مفتوح"
# encoded_input = nlp(input_sequence)
# encoded_input = combine_sub_words(encoded_input)
# # Print each input token with the label
# for word in encoded_input:
#     print(word)
#     # print("{} has label {}".format(word["word"], word["entity"]))



In [221]:

def word2features(sent,pos, i):
    word = sent[i]
    features = {
        'bias': 1.0,
        'word': word,
        'word.ispunct()': is_punct(word),
        'word.isnumeric()': word.isnumeric(),
        'word.isdigit()': word.isdigit(),
        'word.isalpha()': word.isalpha(),
        'word_len': len(word),
        'word_prefix1': word[:1],
        'word_prefix2': word[:2],
        'word_prefix3': word[:3],
        'word_prefix4': word[:4],
        'word_suffix1': word[-1:],
        'word_suffix2': word[-2:],
        'word_suffix3': word[-3:],
        'word_suffix4': word[-4:],
        'word_pos': pos[i],
    }
    if i > 0:
        prev_word = sent[i-1]
        features.update({
            'prev_word': prev_word,
            'prev_word.ispunct()': is_punct(prev_word),
            'prev_word.isnumeric()': prev_word.isnumeric(),
            'prev_word.isdigit()': prev_word.isdigit(),
            'prev_word.isalpha()': prev_word.isalpha(),
            'prev_word_suffix1': prev_word[-1:],
            'prev_word_suffix2': prev_word[-2:],
            'prev_word_suffix3': prev_word[-3:],
            'prev_word_suffix4': prev_word[-4:],
            # 'prev_word_barr': is_it_bar(prev_word),
            # 'prev_word_stat': is_it_status(prev_word),
            'prev_word_pos': pos[i-1],
        })
    if i < len(sent)-1:
        next_word = sent[i+1]
        features.update({
            'next_word': next_word,
            'next_word.ispunct()': is_punct(next_word),
            'next_word.isnumeric()': next_word.isnumeric(),
            'next_word.isdigit()': next_word.isdigit(),
            'next_word.isalpha()': next_word.isalpha(),
            'next_word_prefix1': next_word[:1],
            'next_word_prefix2': next_word[:2],
            'next_word_prefix3': next_word[:3],
            'next_word_prefix4': next_word[:4],
            # 'next_word_barr': is_it_bar(next_word),
            # 'next_word_stat': is_it_status(next_word),
            'next_word_pos': pos[i+1]

        })
    return features


In [222]:
def sent2features(sent):
    input = " ".join(sent)
    encoded_input = nlp(input)
    encoded_input = combine_sub_words(encoded_input)
    pos = [word["entity"] for word in encoded_input]
    
    res = [word2features(sent,pos, i) for i in range(len(sent))]
    if not res:
        print(f"res is empty {sent}")
    return res


def sent2labels(sent):
    return list(sent)


In [223]:
X_train_features = [sent2features(sent.split()) for sent in X_train]
y_train_labels = [sent.split() for sent in y_train]

X_test_features = [sent2features(sent.split()) for sent in X_test]
y_test_labels = [sent.split() for sent in y_test]

In [224]:
X_train_features[0]

[{'bias': 1.0,
  'word': 'مفرق',
  'word.ispunct()': False,
  'word.isnumeric()': False,
  'word.isdigit()': False,
  'word.isalpha()': True,
  'word_len': 4,
  'word_prefix1': 'م',
  'word_prefix2': 'مف',
  'word_prefix3': 'مفر',
  'word_prefix4': 'مفرق',
  'word_suffix1': 'ق',
  'word_suffix2': 'رق',
  'word_suffix3': 'فرق',
  'word_suffix4': 'مفرق',
  'word_pos': 'adj',
  'next_word': 'الطنيب',
  'next_word.ispunct()': False,
  'next_word.isnumeric()': False,
  'next_word.isdigit()': False,
  'next_word.isalpha()': True,
  'next_word_prefix1': 'ا',
  'next_word_prefix2': 'ال',
  'next_word_prefix3': 'الط',
  'next_word_prefix4': 'الطن',
  'next_word_pos': 'noun'},
 {'bias': 1.0,
  'word': 'الطنيب',
  'word.ispunct()': False,
  'word.isnumeric()': False,
  'word.isdigit()': False,
  'word.isalpha()': True,
  'word_len': 6,
  'word_prefix1': 'ا',
  'word_prefix2': 'ال',
  'word_prefix3': 'الط',
  'word_prefix4': 'الطن',
  'word_suffix1': 'ب',
  'word_suffix2': 'يب',
  'word_suffix3': 

In [225]:
crf_model = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
)

In [226]:
# check if any of X_train_features is empty
indexes = []
for i in range(len(X_train_features)):
    if len(X_train_features[i]) == 0:
        print(X_train_features[i]) # why is it empty?
        
# # get the corresponding rows in df 
# df.iloc[indexes]

In [227]:
try:
    crf_model.fit(X_train_features, y_train_labels)
except:
    print("error")
    

In [228]:
# print the accuracy on the training set
print("training accuracy: ", crf_model.score(X_train_features, y_train_labels))
# print the accuracy on the test set
print("test accuracy: ", crf_model.score(X_test_features, y_test_labels))

training accuracy:  0.9999812120718701
test accuracy:  0.998337816576156


In [229]:
y_pred = crf_model.predict(X_test_features)

In [230]:
from sklearn.metrics import accuracy_score

# Flatten the predictions and ground truth labels
y_pred_flat = [tag for sent in y_pred for tag in sent]
y_test_flat = [tag for sent in y_test_labels for tag in sent]

# Calculate the accuracy
accuracy = accuracy_score(y_test_flat, y_pred_flat)

print(f"Accuracy: {accuracy}")


Accuracy: 0.998337816576156


In [241]:
new_text = "طريق بيت شسيءؤئ مسكر"
new_text_features = sent2features(new_text.split())
new_text_pred = crf_model.predict([new_text_features])[0]
new_text_pred

['Barr', 'B-LOC', 'O', 'STAT']

In [243]:
# save the model
import joblib
joblib.dump(crf_model, "../models/NER_CRF.pkl")

In [ ]:
# predict for df
